<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/My_Test/my_test_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [27]:
a = tf.constant([[20., 30., 40., 80.],
          [20., 30., 40., 80.]], dtype=tf.float32)
print(a)
b = tf.keras.layers.Rescaling(1.0 / 4)(a)
c = tf.keras.layers.Flatten()(b)
print(c)


tf.Tensor(
[[20. 30. 40. 80.]
 [20. 30. 40. 80.]], shape=(2, 4), dtype=float32)
tf.Tensor(
[[ 5.   7.5 10.  20. ]
 [ 5.   7.5 10.  20. ]], shape=(2, 4), dtype=float32)


In [28]:
inputs = keras.Input(shape=(28, 28)) # 输入层

# 将像素值缩放到0-1之间
x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)

# 把shape=(28, 28)的展开
x = tf.keras.layers.Flatten()(x)

# 添加层，激活函数为relu
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

# 最后加一个softmax层
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

# 结合inputs, outputs生成模型
model = tf.keras.Model(inputs, outputs)

# 通过summary()得到网络的详细结构
model.summary()

# 模型组合好后就是编译了
# 在编译中，添加损失函数和优化器
# 损失函数用来计算损失并为计算梯度做准备
# 然后通过优化器来更新权重
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# 接下来就是训练模型了
# 这里需要声明训练的轮数及epochs的值
# 以及定义batch_size大小来把大的数据集分成多个batch
# 及更新一次权重需要训练一个batch_size
batch_size = 64
print("Fit on Numpy data")
# 先用numpy类型的数据，训练一个epoch
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

# 再用dataset集合来训练一个epoch
# 生成dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
print("Fit on Dataset")
model.fit(dataset, epochs=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 rescaling_19 (Rescaling)    (None, 28, 28)            0         
                                                                 
 flatten_16 (Flatten)        (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               100480    
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118282 (

In [30]:
# 上述训练中我们把训练过程赋给了一个history变量
# history.history返回训练之后得到的各个指标的值
print(history.history)

{'loss': [0.2648496627807617]}


In [31]:
# 上述训练过程中我们只看的loss这个指标值，我们还可以增加显示的指标值
# 注，我们还可以再次编译训练之后的模型，因为训练之后的模型只不过是权重变了，其他的和最初的模型没啥区别
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=[keras.metrics.SparseCategoricalAccuracy(name='acc')],
)
history = model.fit(dataset, epochs=1)

938/938 [==============================] - 5s 4ms/step - loss: 0.0804 - acc: 0.9754


In [32]:
# 我们还可以将验证集传入来观察模型在验证集上的指标
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
model.fit(dataset, epochs=1, validation_data=val_dataset)

938/938 [==============================] - 6s 6ms/step - loss: 0.0557 - acc: 0.9830 - val_loss: 0.0883 - val_acc: 0.9753


In [33]:
# 好了上述我们用了训练集和验证集，同时让模型在训练的时候显示一些指标
# 但是，问题还远远没有结束，想想还有什么问题，其实到这里我们已经学会训练神经网络了
# 但但但但但是，如果我们在训练的时候遇到了停电，等突然使训练中断怎么办
# 也许你会说，再重新开始训练呗，。。。。。。，如果你已经训练了好几天了，这时候你有什么想法
# 去死吧。。。。。。
# 大型的神经网络训练一般时间是按小时或者天来计算，这时保存已经训练的模型非常重要
# 通过回调模型来设置什么时候保存，如一个batch后，一个epochs后
# 将这个回调模型放在fit()中


# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         filepath='path',
#         save_freq='epoch'
#     )
# ]
# model.fit(dataset, epochs=2, callbacks=callbacks)